In [2]:
import json
import time
from datetime import datetime
import random
import threading
import os

class MixingSimulation:
    def __init__(self):
        self.Volume = 200  # Total volume in Litres
        self.AM_ratio = 0.495
        self.PVDF_ratio = 0.05
        self.CB_ratio = 0.045
        self.NMP_ratio = 0.41

        # Density of each component (g/cm³)
        self.RHO_AM = 2.26
        self.RHO_CB = 1.8
        self.RHO_PVDF = 1.78
        self.RHO_NMP = 1

        # Empirical values of each component
        self.a = 0.9  # AM
        self.b = 2.5  # Binder
        self.c = 0.3  # CB
        self.s = -0.5 # Solvent

        # Initial component volumes
        self.NMP = self.NMP_ratio * self.Volume
        self.PVDF = 0
        self.CB = 0
        self.AM = 0

        self.results = []
        self.total_time = 0

    def run(self, step_percent=0.02, pause_sec=1):
        self._add_in_steps('PVDF', self.PVDF_ratio, step_percent, pause_sec, "PVDF")
        self._add_in_steps('CB', self.CB_ratio, step_percent, pause_sec, "Carbon Black")
        self._add_in_steps('AM', self.AM_ratio, step_percent, pause_sec, "Active Material")

    def _add_in_steps(self, component, ratio, step_percent, pause_sec, label):
        total_volume_to_add = ratio * self.Volume
        step_volume = step_percent * total_volume_to_add
        num_steps = int(1 / step_percent)

        last_saved_time = time.time()

        for _ in range(num_steps):
            self.total_time += pause_sec
            setattr(self, component, getattr(self, component) + step_volume)

            timestamp = datetime.now().isoformat()
            temperature = round(random.uniform(20, 25), 2)
            pressure = round(random.uniform(1, 2), 2)
            rpm = random.randint(300, 600)
            density = self._calculate_density()
            viscosity = self._calculate_viscosity()
            yield_stress = self._calculate_yield_stress()

            result = {
                "TimeStamp": timestamp,
                "Duration": round(self.total_time, 5),
                "AM": round(self.AM, 3),
                "CB": round(self.CB, 3),
                "PVDF": round(self.PVDF, 3),
                "NMP": round(self.NMP, 3),
                "Temperature": temperature,
                "Pressure": pressure,
                "Speed (RPM)": rpm,
                "Density": round(density, 4),
                "Viscosity": round(viscosity, 2),
                "Yield Stress": round(yield_stress, 2)
            }
            self.results.append(result)

            now = time.time()
            if now - last_saved_time >= 10:
                os.makedirs("simulation_output", exist_ok=True)
                filename = f"simulation_output/result_at_{round(self.total_time)}s.json"
                try:
                    with open(filename, "w") as f:
                        json.dump(result, f, indent=4)
                except Exception as e:
                    print(f"Error writing result to file: {e}")
                last_saved_time = now

            time.sleep(pause_sec)

    def _calculate_density(self):
        m_NMP = self.NMP * self.RHO_NMP
        m_PVDF = self.PVDF * self.RHO_PVDF
        m_CB = self.CB * self.RHO_CB
        m_AM = self.AM * self.RHO_AM

        total_mass = m_NMP + m_PVDF + m_CB + m_AM
        total_volume = self.NMP + self.PVDF + self.CB + self.AM
        return total_mass / total_volume if total_volume > 0 else 0

    def _calculate_viscosity(self, max_solid_fraction=0.63, intrinsic_viscosity=3):
        total_volume = self.NMP + self.PVDF + self.CB + self.AM
        if total_volume == 0:
            return 0

        solid_volume = self.PVDF + self.CB + self.AM
        phi = solid_volume / total_volume
        if phi >= max_solid_fraction:
            phi = max_solid_fraction - 0.001

        try:
            viscosity = (1 - (phi / max_solid_fraction)) ** (-intrinsic_viscosity * max_solid_fraction)
            return viscosity * 2
        except Exception as e:
            print(f"Viscosity calculation error: {e}")
            return float('inf')

    def _calculate_yield_stress(self):
        m_NMP = self.NMP * self.RHO_NMP
        m_PVDF = self.PVDF * self.RHO_PVDF
        m_CB = self.CB * self.RHO_CB
        m_AM = self.AM * self.RHO_AM

        y_AM = self.a * m_AM
        y_PVDF = self.b * m_PVDF
        y_CB = self.c * m_CB
        y_NMP = self.s * m_NMP
        return y_NMP + y_PVDF + y_CB + y_AM

    def save_to_json(self, filename="simulation_output/final_results.json"):
        os.makedirs("simulation_output", exist_ok=True)
        try:
            with open(filename, "w") as f:
                json.dump(self.results, f, indent=4)
        except Exception as e:
            print(f"Failed to save final results: {e}")

    def monitor(sim, interval=10):
        while True:
            time.sleep(interval)
            if sim.results:
                latest = sim.results[-1]
                print(f"[{latest['TimeStamp']}] Duration: {latest['Duration']}s | "
                    f"AM: {latest['AM']}L | CB: {latest['CB']}L | PVDF: {latest['PVDF']}L | "
                    f"NMP: {latest['NMP']}L | Temp: {latest['Temperature']}°C | "
                    f"RPM: {latest['Speed (RPM)']} | Viscosity: {latest['Viscosity']} | "
                    f"Yield Stress: {latest['Yield Stress']}")
            if sim.AM >= sim.AM_ratio * sim.Volume:
                break

if __name__ == "__main__":
    sim = MixingSimulation()
    monitor_thread = threading.Thread(target=sim.monitor, args=(10,))
    monitor_thread.start()
    sim.run()
    sim.save_to_json()
    monitor_thread.join()


[2025-05-04T21:57:40.090895] Duration: 10s | AM: 0L | CB: 0L | PVDF: 2.0L | NMP: 82.0L | Temp: 23.08°C | RPM: 371 | Viscosity: 2.15 | Yield Stress: -32.1
[2025-05-04T21:57:50.108816] Duration: 20s | AM: 0L | CB: 0L | PVDF: 4.0L | NMP: 82.0L | Temp: 24.28°C | RPM: 412 | Viscosity: 2.31 | Yield Stress: -23.2
[2025-05-04T21:58:00.114168] Duration: 30s | AM: 0L | CB: 0L | PVDF: 6.0L | NMP: 82.0L | Temp: 23.52°C | RPM: 505 | Viscosity: 2.48 | Yield Stress: -14.3
[2025-05-04T21:58:10.118047] Duration: 40s | AM: 0L | CB: 0L | PVDF: 8.0L | NMP: 82.0L | Temp: 20.1°C | RPM: 527 | Viscosity: 2.67 | Yield Stress: -5.4
[2025-05-04T21:58:20.123510] Duration: 50s | AM: 0L | CB: 0L | PVDF: 10.0L | NMP: 82.0L | Temp: 24.41°C | RPM: 467 | Viscosity: 2.86 | Yield Stress: 3.5
[2025-05-04T21:58:30.127818] Duration: 60s | AM: 0L | CB: 1.8L | PVDF: 10.0L | NMP: 82.0L | Temp: 22.41°C | RPM: 321 | Viscosity: 3.05 | Yield Stress: 4.47
[2025-05-04T21:58:40.132534] Duration: 70s | AM: 0L | CB: 3.6L | PVDF: 10.0L 